# Log 
1. Saving features and labels to avoid running them through the covnet repeatedly
2. the loaded data seems to produce the same results, so we can proceed


In [1]:
# print out data splits for all categories
import keras, os, shutil
#keras.__version__ 2.4.3


base = "data"
splits = ["training", "validating", "testing"]
categories = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
target_size = 128


for split in splits:
    for category in categories:
        path = os.path.join(base, split, category)
        print (path, "number of items:", len(os.listdir(path)))

data/training/cardboard number of items: 199
data/training/glass number of items: 199
data/training/metal number of items: 199
data/training/paper number of items: 199
data/training/plastic number of items: 299
data/training/trash number of items: 69
data/validating/cardboard number of items: 104
data/validating/glass number of items: 202
data/validating/metal number of items: 111
data/validating/paper number of items: 295
data/validating/plastic number of items: 183
data/validating/trash number of items: 38
data/testing/cardboard number of items: 100
data/testing/glass number of items: 100
data/testing/metal number of items: 100
data/testing/paper number of items: 100
data/testing/plastic number of items: 100
data/testing/trash number of items: 30


In [2]:
# import the VGG16 model
import numpy as np
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(target_size, target_size, 3))

conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [3]:
#load data and extract features
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count, 6)) #added '6' to fit the number of classes
    generator = datagen.flow_from_directory(
        directory,
        target_size=(target_size, target_size),
        batch_size= batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features("data/training", 1160)#changed to fit sample size
validation_features, validation_labels = extract_features("data/validating", 920) #this one too
test_features, test_labels = extract_features("data/testing", 520) # and this

# reshape
train_features = np.reshape(train_features, (1160, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (920, 4 * 4 * 512))
test_features = np.reshape(test_features, (520, 4 * 4 * 512))

Found 1164 images belonging to 6 classes.
Found 933 images belonging to 6 classes.
Found 530 images belonging to 6 classes.


In [9]:
# save features and labels
np.save("saved_train_features.npy", train_features);
np.save("saved_validation_features.npy", validation_features);
np.save("saved_test_features.npy", test_features);
np.save("saved_train_labels.npy", train_labels);
np.save("saved_validation_labels.npy", validation_labels);
np.save("saved_test_labels.npy", test_features);



In [11]:
#  test load features and labels
loaded_train_features = np.load("saved_train_features.npy");
loaded_validation_features = np.load("saved_validation_features.npy");
loaded_test_features = np.load("saved_test_features.npy");
loaded_train_labels = np.load("saved_train_labels.npy");
loaded_validation_labels = np.load("saved_validation_labels.npy");
loaded_test_labels = np.load("saved_test_labels.npy");

In [12]:
# TESTING IF THE LOADED DATA WORKS!
# build network and print summary
from keras import layers
from keras import models
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(128, kernel_regularizer=regularizers.l2(0.001), activation='relu', input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

model.summary()


# add optimizer
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 1,049,478
Trainable params: 1,049,478
Non-trainable params: 0
_________________________________________________________________


In [13]:
# train and save model
history = model.fit(loaded_train_features, loaded_train_labels,
                    epochs=10,
                    batch_size=20,
                    validation_data=(loaded_validation_features, loaded_validation_labels))

#model.save('trash-classifier-0-6.h5')

Epoch 1/10
58/58 [==============================] - 2s 37ms/step - loss: 1.8167 - acc: 0.3431 - val_loss: 1.4750 - val_acc: 0.5620
Epoch 2/10
58/58 [==============================] - 2s 30ms/step - loss: 1.4456 - acc: 0.5138 - val_loss: 1.3607 - val_acc: 0.5500
Epoch 3/10
58/58 [==============================] - 2s 37ms/step - loss: 1.2159 - acc: 0.6086 - val_loss: 1.2250 - val_acc: 0.5913
Epoch 4/10
58/58 [==============================] - 2s 36ms/step - loss: 1.1046 - acc: 0.6655 - val_loss: 1.1323 - val_acc: 0.6522
Epoch 5/10
58/58 [==============================] - 3s 46ms/step - loss: 0.9819 - acc: 0.7233 - val_loss: 1.0188 - val_acc: 0.6957
Epoch 6/10
58/58 [==============================] - 2s 36ms/step - loss: 0.9191 - acc: 0.7336 - val_loss: 0.9941 - val_acc: 0.6772
Epoch 7/10
58/58 [==============================] - 3s 45ms/step - loss: 0.8397 - acc: 0.7534 - val_loss: 1.0476 - val_acc: 0.6576
Epoch 8/10
58/58 [==============================] - 3s 52ms/step - loss: 0.7668 - a

In [14]:
# plot accuracy and loss graphs
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()